In [9]:
import pyvista as pv
from seagullmesh import Mesh3
import numpy as np
import pymeshfix
from ga_population import *
import numpy as np
from sklearn.decomposition import PCA

pm_monkey = ProbeMesh.load(
    mesh_file=r"D:\mesh_extract_work\VervertMonkey\JF0N2N0B_VervetMonkeys_Blender\JF0N2N0B_VervetMonkeys_Blender\JF0N2N0B_VervetMonkeys_Run_v4_mesh_extraction.ply",
    corpus_index='JC0L219A2_GreyTabbyCat_Trot_v4_orig_Main.png',
    cam_pos=(-0.883204, -1.11311, 1.03748),
    tgt_pos=(0.062604, 0.152099, 0.187549),
    repair=True,
    fill_holes=True,
    cache=True,
    recache=False,
    wrap=True,
    relative_alpha=600,
    relative_offset=1000,
    simplify_n_faces=3000,
)

pm_cat = ProbeMesh.load(
    mesh_file=r"D:\mesh_extract_work\GreyTabbyCat\JC0L219A2_GreyTabbyCat_Blender\JC0L219A2_GreyTabbyCat_Blender\JC0L219A2_GreyTabbyCat_Trot_v4.mesh_extraction2.ply",
    corpus_index='JF0N2N0B_VervetMonkeys_Run_v4_orig_Main.png',
    cam_pos=(-0.19411747, -0.86916119,  0.33897164),
    tgt_pos=( 2.28889287e-04, -3.71107072e-01,  1.14519626e-01),
    repair=True,
    fill_holes=True,
    cache=True,
    recache=False,
    wrap=True,
    relative_alpha=100,
    relative_offset=600,
    simplify_n_faces=3000,
) 

pm_hare = ProbeMesh.load(
    mesh_file=r"D:\mesh_extract_work\SnowshoeHare\JU0L219A1_SnowshoeHare_Summer_Blender\JU0L219A1_SnowshoeHare_Summer_Jump_v4_A_mesh_extraction.ply",
    corpus_index='JU0L219A1_SnowshoeHare_Summer_Jump_v4_A_orig_Main.png',
    cam_pos=(-0.52659595, -0.80872011,  0.40757233),
    tgt_pos=(0.019130593165755272, -0.4574986398220062, 0.13628436625003815),
    repair=True,
    fill_holes=True,
    cache=True,
    recache=False,
    wrap=True,
    relative_alpha=600,
    relative_offset=1000,
    simplify_n_faces=3000,
) 

pms = [pm_monkey, pm_cat, pm_hare]

In [8]:
# pm.mesh.plot(show_edges=True)

In [21]:
# models = self = SourceModels.current_trained()
# pm_weights = models.get_weights(pms, expt_kwargs=dict(outputs_at='vertices'))
# torch.save(pm_weights, 'temp_weights.pt')

pm_weights = torch.load('temp_weights.pt')

In [15]:
for pm, weights in zip(pms, pm_weights):
    pm.plot_weights(weights=weights.pca(), shape=(1, 3), window_size=(1500, 500), scalar_bar=True, titles=['PC1', 'PC2', 'PC3'], render=False)

Widget(value='<iframe src="http://localhost:56009/index.html?ui=P_0x2b2d4633430_7&reconnect=auto" class="pyvis…

Widget(value='<iframe src="http://localhost:56009/index.html?ui=P_0x2b28cc1dcf0_8&reconnect=auto" class="pyvis…

Widget(value='<iframe src="http://localhost:56009/index.html?ui=P_0x2b28d481960_9&reconnect=auto" class="pyvis…

In [25]:
plotter = pv.Plotter(shape=(1, 3))
for i, (pm, weights) in enumerate(zip(pms, pm_weights)):
    plotter.subplot(0, i)
    plotter.camera = pm.camera
    plotter.add_mesh(pm.mesh, scalars=weights.pca(), rgb=True)
plotter.show()

Widget(value='<iframe src="http://localhost:56009/index.html?ui=P_0x2b28d2529e0_11&reconnect=auto" class="pyvi…